In [288]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [289]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [290]:
## Read Data for the Fraudulent Email Kaggle Challenge
train = pd.read_csv("data/kg_train.csv",encoding='latin-1')
test = pd.read_csv("data/kg_test.csv",encoding='latin-1')
# Reduce the training set to speead up development. 
# Modify for final system
train = train.head(1000)
test = test.head(1000)
print(train.shape)
print(test.shape)
train.fillna("",inplace=True)
test.fillna("",inplace=True)

(1000, 2)
(1000, 1)


In [291]:
train.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [292]:
test.head()

,text
0,usiness is for the fact that the deceased man ...
1,They are happy to adjust to the afternoon. I a...
2,Lael Brainard was confirmed 78-19 this afterno...
3,H <hrod17@clintonemail.com>Friday March 26 201...
4,"n;""> Dear Good Friend,<br><br><br>I am happy t..."


### Let's divide the training and test set into two partitions

In [293]:
# We already have training and testing into two partitions

In [294]:
train['text']


0      DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                               Will do.
2      Nora--Cheryl has emailed dozens of memos about...
3      Dear Sir=2FMadam=2C I know that this proposal ...
4                                                    fyi
                             ...                        
995    So what's the latest? It sounds contradictory ...
996    TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997    Barb I will call to explain. Are you back in t...
998      Yang on travelNot free tonite.May work tomorrow
999    sbwhoeopSunday February 21 2010 7:42 PMHShaunH...
Name: text, Length: 1000, dtype: object

In [295]:
test['text']

0      usiness is for the fact that the deceased man ...
1      They are happy to adjust to the afternoon. I a...
2      Lael Brainard was confirmed 78-19 this afterno...
3      H <hrod17@clintonemail.com>Friday March 26 201...
4      n;"> Dear Good Friend,<br><br><br>I am happy t...
                             ...                        
995    Dear Friend,Do accept my sincere apologies if ...
996    FROM THE DESK OF:MR.WANG QINHANG SENG BANK LTD...
997    He can speak at 7pm. He said that it does not ...
998    Mary Landrieu asked me to attend a dinner abou...
999    Oscar should have printed gates shangrila conf...
Name: text, Length: 1000, dtype: object

## Data Preprocessing

In [296]:
import re
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [297]:
def clean_html(html):
    """Copied from NLTK package.
    Remove HTML markup from the given string.

    :param html: the HTML string to be cleaned
    :type html: str
    :rtype: str"""
    # First we remove inline JavaScript/CSS:
    cleaned = html.strip()
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", " ", cleaned)
    
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", " ", cleaned)
    
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)   
    

    return cleaned.strip()

In [298]:
# Apply clean_html function to each value in the training series
train['text'] = train['text'].map(clean_html)
test['text'] = test['text'].map(clean_html)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [299]:
# let's create a function that does all the above :
def preprocess(pdseries):
    # Define the punctuation characters to remove
    punctuation = string.punctuation
    
    def process_text(text):
        if not isinstance(text, str):
            return text
        
        # Remove special characters and digits
        text = ''.join(char for char in text if char not in punctuation and not char.isdigit())
        
        # Remove all single characters and single characters from the start
        text = ' '.join(word for word in text.split() if len(word) > 1)
        
        # Substitute multiple spaces with a single space and remove prefixed 'b'
        text = ' '.join(text.split()).lstrip('b')
        
        # Convert to lowercase and remove Unicode characters
        text = text.lower()
        text = ''.join(char for char in text if char.isalnum() or char.isspace())
        
        return text
    
    # Apply the process_text function to each element in the series
    return pdseries.map(process_text)

In [300]:
train['text'] = preprocess(train['text'])
test['text'] = preprocess(test['text'])

In [301]:
train['text']

0      dear sir strictly private business proposal am...
1                                                will do
2      noracheryl has emailed dozens of memos about h...
3      dear sirfmadamc know that this proposal might ...
4                                                    fyi
                             ...                        
995    so whats the latest it sounds contradictory an...
996    transfer of million pounds to youraccountmy na...
997    barb will call to explain are you back in the ...
998       yang on travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh just talked ...
Name: text, Length: 1000, dtype: object

In [302]:
test['text']

0      usiness is for the fact that the deceased man ...
1      they are happy to adjust to the afternoon am g...
2      lael brainard was confirmed this afternoonmigu...
3      friday march amsbwhoeopãâãâãâ rei have extende...
4      dear good friend am happy to inform you about ...
                             ...                        
995    dear frienddo accept my sincere apologies if m...
996    from the desk ofmrwang qinhang seng bank ltdde...
997    he can speak at pm he said that it does not ha...
998    mary landrieu asked me to attend dinner about ...
999    oscar should have printed gates shangrila conf...
Name: text, Length: 1000, dtype: object

## Now let's work on removing stopwords
Remove the stopwords.

In [303]:
from nltk.corpus import stopwords
print (stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [304]:
stopwords_to_remove = stopwords.words("english")

# Remove stopwords
train['text'] = train['text'].map(lambda x: ' '.join(word for word in x.split() if word.lower() not in stopwords_to_remove) if isinstance(x, str) else x)
test['text'] = test['text'].map(lambda x: ' '.join(word for word in x.split() if word.lower() not in stopwords_to_remove) if isinstance(x, str) else x)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [305]:
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Break sentences into words and lemmatize
train['text'] = train['text'].map(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()) if isinstance(x, str) else x)
test['text'] = test['text'].map(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()) if isinstance(x, str) else x)

In [306]:
train['text']

0      dear sir strictly private business proposal mi...
1                                                       
2      noracheryl emailed dozen memo haiti weekend pl...
3      dear sirfmadamc know proposal might surprise e...
4                                                    fyi
                             ...                        
995    whats latest sound contradictory af decide sha...
996    transfer million pound youraccountmy name mrej...
997                       barb call explain back country
998          yang travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh talked shaun...
Name: text, Length: 1000, dtype: object

In [307]:
test['text']

0      usiness fact deceased man foreigner authorized...
1      happy adjust afternoon going suggest pm start ...
2      lael brainard confirmed afternoonmiguel rodrig...
3      friday march amsbwhoeopãâãâãâ rei extended con...
4      dear good friend happy inform successin gettin...
                             ...                        
995    dear frienddo accept sincere apology mail meet...
996    desk ofmrwang qinhang seng bank ltddes voeux r...
997           speak pm said secure ops connect residence
998    mary landrieu asked attend dinner foster care ...
999    oscar printed gate shangrila conf speechalso p...
Name: text, Length: 1000, dtype: object

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [308]:
from nltk.tokenize import word_tokenize
bag_of_words = train.copy()
# Tokenize sentences
bag_of_words['text'] = bag_of_words['text'].map(lambda x: word_tokenize(x) if isinstance(x, str) else x)

In [309]:
from collections import Counter

# Get the top 10 words for label 0
words_label_0 = [word for sublist in bag_of_words.loc[bag_of_words['label'] == 0, 'text'] for word in sublist]
top_words_label_0 = Counter(words_label_0).most_common(10)

# Get the top 10 words for label 1
words_label_1 = [word for sublist in bag_of_words.loc[bag_of_words['label'] == 1, 'text'] for word in sublist]
top_words_label_1 = Counter(words_label_1).most_common(10)

print("Top 10 words for ham:", top_words_label_0)
print("Top 10 words for spam:", top_words_label_1)

Top 10 words for ham: [('u', 115), ('pm', 115), ('would', 106), ('state', 103), ('president', 94), ('call', 91), ('time', 84), ('percent', 77), ('secretary', 76), ('work', 73)]
Top 10 words for spam: [('money', 926), ('account', 800), ('bank', 748), ('fund', 708), ('u', 553), ('business', 476), ('transaction', 426), ('country', 414), ('transfer', 399), ('million', 388)]


## Extra features

In [310]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train['money_mark'] = train['text'].str.contains(money_simbol_list)*1
train['suspicious_words'] = train['text'].str.contains(suspicious_words)*1
train['text_len'] = train['text'].apply(lambda x: len(x)) 

test['money_mark'] = test['text'].str.contains(money_simbol_list)*1
test['suspicious_words'] = test['text'].str.contains(suspicious_words)*1
test['text_len'] = test['text'].apply(lambda x: len(x)) 

train.head()

,text,label,money_mark,suspicious_words,text_len
0,dear sir strictly private business proposal mi...,1,1,1,1504
1,,0,1,0,0
2,noracheryl emailed dozen memo haiti weekend pl...,0,1,0,110
3,dear sirfmadamc know proposal might surprise e...,1,1,1,1380
4,fyi,0,1,0,3


In [311]:
test.head()

,text,money_mark,suspicious_words,text_len
0,usiness fact deceased man foreigner authorized...,1,1,799
1,happy adjust afternoon going suggest pm start ...,1,0,102
2,lael brainard confirmed afternoonmiguel rodrig...,1,0,155
3,friday march amsbwhoeopãâãâãâ rei extended con...,1,0,51
4,dear good friend happy inform successin gettin...,1,1,807


## How would the Bag of Words with Count Vectorizer concept work?

In [312]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the Count Vectorizers
vectorizer_train = CountVectorizer()
vectorizer_test = CountVectorizer()

# Fit and transform the corpus
X = vectorizer_train.fit_transform(train['text'])
y = vectorizer_test.fit_transform(test['text'])

# Convert the results to DataFrames for easy manipulation
vectorized_df_train = pd.DataFrame(X.toarray(), columns=vectorizer_train.get_feature_names_out())
vectorized_df_test = pd.DataFrame(y.toarray(), columns=vectorizer_test.get_feature_names_out())

In [313]:
vectorized_df_train

,aac,aaclocated,aae,aag,aaronovitchon,abacha,abachabefore,abachac,abachace,abachaco,...,ãââolic,ãââ½,ãââ½trangers,ãââ½ãââ½,ãââ½ãââ½b,ãââ½ãââ½c,ãââ½ãââ½d,ãââ½ãââ½for,ãââ½ãââ½to,ãââ½ãââ½v
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [314]:
vectorized_df_test

,aa,aabaibqblbbadaag,aacfcaadf,aaeefaafbedccc,aafiasiddiqui,aakamichelciyahoofrto,aallensmithfyahooeca,aam,aba,abacha,...,ãâãâãâ½,ãâãâãâ½million,ãâãâãâ½millionthis,ãâãâãâ½min,ãâãâãâ½minute,ãâãâãâ½nãâãâãâ½ãâãâãâ½aãâãâãâ½sãâãâãâ½ãâãâãâ½ãâãâãâ½wãâãâãâ½ãâãâãâ½aãâãâãâ½aãâãâãâ½ãâãâãâ½ãâãâãâ½bãâãâãâ½ãâãâãâãâãâ½ãâãâãâ½iãâãâãâ½hãâãâãâ½dãâãâãâ½uãâãâãâ½tãâãâãâ½ãâãâãâ½ãâãâãâ½ãâãâãâ½ãâãâãâ½aãâãâãâ½aãâãâãâ½ãâãâãâ½aãâãâãâ½wãâãâãâ½ãâãâãâ½ãâãânãâãâãâ½ãâãâãâ½ãâãâyãâãâãâ½ãâãâãâãâãâ½aãâãâãâ½ãâãâãâ½ãâãâãâãâãâãâãâãâãâ½ãâãâãâ½nãâãâãâ½ãâãâãâ½ãâãâãâ½ãâãâãâ½ãâãâãâ½c,ãâãâãâ½tranger,ãâãâãâ½tre,ãâãâãâ½tãâãâãâ½,ãâãâãâ½ãâãâãâ½uãâãâãâ½tãâãâãâ½ãâãâãâ½
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [317]:
# Initialize TF-IDF vectorizer instances
vect_train = TfidfVectorizer()
vect_test = TfidfVectorizer()

# Fit and transform each series separately
tfidf_train = vect_train.fit_transform(train['text'])
tfidf_test = vect_test.fit_transform(test['text'])

# Print the shape of the resulting vectorized datasets
print("Shape of TF-IDF matrix for series1:", tfidf_train.shape)
print("Shape of TF-IDF matrix for series2:", tfidf_test.shape)

Shape of TF-IDF matrix for series1: (1000, 19168)
Shape of TF-IDF matrix for series2: (1000, 19593)


In [318]:
# Optionally, convert to DataFrames for inspection
df_tfidf_train = pd.DataFrame(tfidf_train.toarray(), columns=vect_train.get_feature_names_out())
df_tfidf_test = pd.DataFrame(tfidf_test.toarray(), columns=vect_test.get_feature_names_out())

# Display the DataFrames
print("\nTF-IDF DataFrame for series1:")
print(df_tfidf_train)
print("\nTF-IDF DataFrame for series2:")
print(df_tfidf_test)


TF-IDF DataFrame for series1:
     aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
1    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
2    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
3    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
4    0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
..   ...         ...  ...  ...            ...     ...           ...      ...   
995  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
996  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
997  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
998  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
999  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   

     aba

## And the Train a Classifier?

In [320]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# X has been vectorised using TF-IDF in the cells above
X = tfidf_train
y = train['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Multinomial Naive Bayes classifier
clf = MultinomialNB()

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.945
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.92      0.95       125
           1       0.88      0.99      0.93        75

    accuracy                           0.94       200
   macro avg       0.94      0.95      0.94       200
weighted avg       0.95      0.94      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code